In [ ]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv 

A partir de la API publica de yelp, puedo realizar ingesta de datos referente a los locales y sus cacacteristicas junto a reviews de estos.

## Business Yelp

Ingesta de de los locales que son restaurantes y pertencen a los estados trabajados.


In [ ]:

load_dotenv('../extras/.env') # Cargo ela archivo donde esta la variable de entorno.
api_key_yelp =  os.getenv("API_KEY_YELP") # Cargo la variable de entorno

In [47]:
def state_normalize(state):
    if state == 'CA':
        return 'California'
    elif state == 'FL':
        return 'Florida'
    elif state == 'NJ':
        return 'New Jersey'
    elif state == 'IL':
        return 'Illinois'

In [35]:
def business(state):
    url = f'https://api.yelp.com/v3/businesses/search'

    params = {
        'location': state,
        'categories':','.join(['restaurant','Restaurant','restaurants','Restaurants']),
        'limit':50
    }

    headers = {
        'Authorization': f'Bearer {api_key_yelp}',
        'accept': 'application/json'
    }

    response = requests.get(url, headers=headers,params=params)

    if response.status_code == 200:
        data = response.json()
        # Convierte el JSON en un DataFrame de pandas
        #df = pd.json_normalize(data)
        businesses = pd.json_normalize(data['businesses'])
        return businesses
    else:
        print(f'Error en la solicitud. Código de estado: {response.status_code}')
        print(response.text)

In [36]:
url = f'https://api.yelp.com/v3/businesses/search'

yelp_bussines = pd.DataFrame()
for state in ['CA','FL','NJ','IL']:
    businesses = business(state)
    yelp_bussines = pd.concat([businesses,yelp_bussines])

In [37]:
yelp_bussines.shape[0]


200

Como puede verse cargue datos de locales para los estados a usar, pero solo se ingestar 200 locales, y se consumieron 4 consultas de las 500

Voy a realizar un etl para dejar los datos, igual a los del Data Wherehouse

In [38]:
yelp_bussines.columns

Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'transactions', 'price', 'phone',
       'display_phone', 'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address'],
      dtype='object')

In [45]:
yelp_bussines['categories'] = yelp_bussines['categories'].apply(lambda x: [item['alias'] for item in x] if isinstance(x, list) else [])

In [48]:
yelp_bussines['state'] = yelp_bussines['state'].apply(state_normalize)

In [49]:
yelp_bussines.rename(columns={
    'id':'business_id',
    'name':'name',
    'coordinates.latitude':'latitude',
    'coordinates.longitude':'longitude',
    'categories':'categories',
    'rating':'stars',
    'location.state':'state',
    
},inplace=True)
columnas = ['business_id', 'name', 'latitude','longitude','categories','stars','state']
yelp_bussines[columnas]

,business_id,name,latitude,longitude,categories,stars,state
0,qjnpkS8yZO8xcyEIy5OU9A,Girl & The Goat,41.884181,-87.647947,"[newamerican, bakeries, coffee]",4.5,Illinois
1,boE4Ahsssqic7o5wQLI04w,The Purple Pig,41.890694,-87.624782,"[tapasmallplates, mediterranean, newamerican]",4.5,Illinois
2,xoi7Cw7FoknAx5p880RtWQ,Au Cheval,41.884660,-87.647668,"[bars, tradamerican]",4.5,Illinois
3,cKZNbMvoqJaUe7n6lf6i7w,Wildberry Pancakes and Cafe,41.884668,-87.622880,"[pancakes, waffles, cafes]",4.5,Illinois
4,riT822EnU7y_5eCuJsd9sA,Cindy's Rooftop,41.881689,-87.625006,"[newamerican, seafood, breakfast_brunch]",4.0,Illinois
...,...,...,...,...,...,...,...
45,PGUM86R57lXPi4LFHJWhAg,Dish Bistro,37.637530,-118.966839,"[burgers, sandwiches, desserts]",5.0,California
46,RxBjn1XKhSTeTZ7bqMVysA,Sam's Italian Deli & Market,36.772659,-119.772999,"[delis, beer_and_wine, intlgrocery]",4.5,California
47,VvPOfhbH6BH3I-vFG1hALQ,Ramen Hayashi,36.837882,-119.685182,[ramen],4.5,California
48,a3mxzNSSW11qjYLFjEbMGA,Rio Acai Bowls - Downtown Fresno,36.742920,-119.800360,"[acaibowls, poke, vegan]",4.5,California


## Reviews Yelp


In [ ]:
def ingest_reviews_yelp(business_id):
    url =f'https://api.yelp.com/v3/businesses/{business_id}/reviews?limit=50&sort_by=yelp_sort"'

    headers = {
        'Authorization': f'Bearer {api_key_yelp}',
        'accept': 'application/json'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        reviews_list = data.get('reviews', [])
        return  pd.json_normalize(reviews_list)

    else:
        print(f'Error en la solicitud. Código de estado: {response.status_code}')


In [51]:
#funcion que retorna una lista con los id que no existan ya en el DW
business_id = 'eEOYSgkmpB90uNA7lDOMRA'
reviews_yelp = ingest_reviews_yelp(business_id)
reviews_yelp['business_id'] = business_id
reviews_yelp.shape[0]

3

Puede verse que cada review se saca a partir de un business id por ende, con 500 consultas que es el limite diario se obtendrian reviews solo para 500 locales.

In [52]:
reviews_yelp.rename({
    'id	':'review_id',
    'user.id':'user_id',
    'business_id':'business_id',
    'sentiment':'sentiment',
    'time_created':'date'
},inplace=True)

,id,url,text,rating,time_created,user.id,user.profile_url,user.image_url,user.name,business_id
0,CE36Dcxv60ibaHntpm94zg,https://www.yelp.com/biz/vietnamese-food-truck...,Vietnamese Food Truck was a super perfect dinn...,5,2021-11-30 16:06:58,qjfMBIZpQT9DDtw_BWCopQ,https://www.yelp.com/user_details?userid=qjfMB...,https://s3-media2.fl.yelpcdn.com/photo/zo5AHLN...,Rachel M.,eEOYSgkmpB90uNA7lDOMRA
1,x9Vsx5gGtUi-LZwk-BREKQ,https://www.yelp.com/biz/vietnamese-food-truck...,"I have traveled to Vietnam several times, love...",5,2022-01-02 17:18:29,01cQC-PaQb-cqbRAvwJU-A,https://www.yelp.com/user_details?userid=01cQC...,https://s3-media4.fl.yelpcdn.com/photo/yQAoMte...,Art K.,eEOYSgkmpB90uNA7lDOMRA
2,WidU0v-65L7sEoLI4NuYCw,https://www.yelp.com/biz/vietnamese-food-truck...,don't come to this food truck. the owner (?) a...,1,2023-05-08 09:53:43,GjGcvFTzg3QD35OwewXdvA,https://www.yelp.com/user_details?userid=GjGcv...,None,Alexandria T.,eEOYSgkmpB90uNA7lDOMRA


In [ ]:
import s3
bucket_nombre = 'projectgoogleyelp'
objeto_key = 'yelp/bussiness_yelp.parquet.gz'
destino_archivo = './asd.parquet.gz'
# Llama a la función para descargar el objeto desde S3
s3.download_file(bucket_nombre,objeto_key,destino_archivo)
